In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# from langchain.vectorstores import FAISS
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, dotenv_values

# from langchain.prompts import PromptTemplate
# from langchain.chains import RetrievalQA
# import asyncio
import yaml
import os
import pandas as pd
import glob
import datetime


# load env token
from os import path

# curpath = path.dirname(os.path.abspath(__file__))

with open("token.yaml", "r") as token_yaml:
    try:
        token = yaml.safe_load(token_yaml)
    except yaml.YAMLError as exc:
        print(exc)
api_key = token["openai_token"]
config = dotenv_values("../.env")
os.environ["OPENAI_API_KEY"] = api_key

db_path = "./chroma_db"
llm_model = ["gpt-3.5-turbo", "gpt-3.5-turbo-1106", "gpt-4-0314"]
chosen_model = llm_model[1]

In [2]:
embeddings_opennai = OpenAIEmbeddings()

# read data and save db
frames = []
for f in glob.glob("data/*.csv"):
    print(f"load file: {f}")
    data = pd.read_csv(f)
    data = data[["id", "title", "contents", "date", "category", "source"]]
    frames.append(data)
df = pd.concat(frames)
df

load file: data/yahoo.csv
load file: data/cnyes.csv
load file: data/moneydj.csv
load file: data/udn.csv
load file: data/moneyudn.csv


,id,title,contents,date,category,source
0,c6040d65-4bfe-4d96-985f-2fc9801b7e3a,《金融股》國票金前3季每股盈餘0.46元,【時報-台北電】國票金(2889)公布前3季財務報告，利息淨收益9.58億元，淨收益54.0...,2023/11/21,最新新聞,yahoo
1,e70082ec-b92b-4e84-860a-b4affdd562cb,《大陸金融》萬達地產到期美元債券 擬延長11個月,【時報-台北電】信報引述交易所公告，萬達地產擬將2024(明)年到期的7.25%美元債券到期...,2023/11/21,最新新聞,yahoo
2,8f6dd49f-8d91-456a-9d26-5333f1e17354,看好明年AST2600滲透率，美系外資重申加碼信驊，目標價3150元,【財訊快報／記者劉居全報導】美系外資在新出爐的報告中表示，看好信驊(5274)營運進入佳境，...,2023/11/21,最新新聞,yahoo
3,d201c108-d2eb-44ba-9522-71524700dadb,中國經濟刺激預期提振，新交所鐵礦砂一度上漲1.46%，升至九個月高點,【財訊快報／劉敏夫】外電報導指出，隨著有關中國最新經濟刺激計畫的更多細節浮出檯面，有關需求的...,2023/11/21,最新新聞,yahoo
4,72d3aebe-030f-4dcc-8c58-2d11c6b2b20d,美股大漲台股7連紅，權值股龍頭蓄勢挑戰今年新高,【財訊快報／方亞申】美股在微軟、輝達上漲帶動下，科技股再度上漲，並帶動道瓊及標普拉升，美元指...,2023/11/21,最新新聞,yahoo
...,...,...,...,...,...,...
892,ce07697b-3045-430e-b563-a778923b9dcd,不是一銀、兆豐 這家公股銀行年終獎金估超過7.3個月,六家上市公股金融機構年終績效獎金紛傳好消息，除了公股獲利王兆豐金（2886）旗下兆豐銀可望上...,2023/11/17,即時金融,moneyudn
893,7e578ef5-eb2b-4b50-9d12-83a086694aef,富邦金揭露產險的防疫險再保部位尚有165億未獲賠付,對於防疫險的國際再保理賠，富邦金今日在法說會上揭露最新數字指出，第三季已再獲16億元的國際再...,2023/11/17,即時金融,moneyudn
894,e3d6d9ef-536b-4781-a06f-e677375c2612,"不畏寒流！ KPMG 志工日逾1,600名同仁熱血參與",不畏入冬第一波寒流，KPMG安侯建業舉辦第16屆企業志工日，今（17）日一早 KPMG安侯建...,2023/11/17,即時金融,moneyudn
895,13a5fcb4-bb36-401a-ae6d-271497c928e1,富邦金：明年股利有機會優於今年 不必動公積配息,富邦金今年前3季累計稅後淨利達新台幣677.8億元，年減2成，但仍居金控業獲利王。富邦金總經...,2023/11/17,即時金融,moneyudn


In [3]:
# load data
loader = DataFrameLoader(df, page_content_column="contents")
# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
documents = text_splitter.split_documents(loader.load())
documents

[Document(page_content='【時報-台北電】國票金(2889)公布前3季財務報告，利息淨收益9.58億元，淨收益54.07億元，稅前淨利24.46億元，本期淨利19.39億元，歸屬於母公司業主淨利15.94億元，基本每股盈餘0.46元。（編輯：邱致馨）', metadata={'id': 'c6040d65-4bfe-4d96-985f-2fc9801b7e3a', 'title': '《金融股》國票金前3季每股盈餘0.46元', 'date': '2023/11/21', 'category': '最新新聞', 'source': 'yahoo'}),
 Document(page_content='【時報-台北電】信報引述交易所公告，萬達地產擬將2024(明)年到期的7.25%美元債券到期日延長至年底12月29日，共延長11個月，本金額為6億美元。', metadata={'id': 'e70082ec-b92b-4e84-860a-b4affdd562cb', 'title': '《大陸金融》萬達地產到期美元債券 擬延長11個月', 'date': '2023/11/21', 'category': '最新新聞', 'source': 'yahoo'}),
 Document(page_content='【財訊快報／記者劉居全報導】美系外資在新出爐的報告中表示，看好信驊(5274)營運進入佳境，AST2600滲透率將一路成長到2024年，重申「加碼」評等，目標價維持3150元。美系外資指出，基於AI(人工智慧)需求強勁，加上英特爾Eagle Stream帶動AST2600拉貨，現階段客戶庫存更加健康，目前信驊B/B(Book to bill ratio)比率接近1，預估信驊第四季營收兩位數成長不變。\n展望第四季，美系外資表示，預估信驊出貨量將恢復到歷史正常水準，而伺服器總出貨量也應該趨穩，主要是人工智慧伺服器仍佔據雲端資本支出很大一部分。至於中國大陸市場需求部分，預估到2024年需求仍疲軟。\n此外，美系外資表示，看好AST2600滲透率將一路成長到2024年，但新的BMC(遠端控制晶片)拉動主要集中在AST2600。信驊有機會在美國主要市場奪回一些超大規模企業市佔率，持續擴展美國企業客戶。', metadata={'id': '8f6d

In [9]:
len(str(documents))

1679500

In [12]:
import math
import time
import tiktoken

def num_tokens_from_string(string: str, chosen_model: str) -> int:
    """Returns the number of tokens in a text string."""
    # encoding = tiktoken.get_encoding(encoding_name) # ex. encodeing_name = "cl100k_base"
    encoding = tiktoken.encoding_for_model(chosen_model) #automatically load the correct encoding for a given model name
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [13]:
# Limitation of model text-embedding-ada-002 is 1,000,000 TPM
num_tokens_from_string(str(documents), chosen_model)

1917272

In [45]:
total_tokens = num_tokens_from_string(str(documents), chosen_model)
token_chunk_size=800000 # buffer for limitation 1,000,000 

if total_tokens>1000000:
    dividend_num = math.ceil(total_tokens/token_chunk_size)
    text_chunk_size = math.ceil(len(documents) / dividend_num )
    print(f"chunk documents to tokenize | chunk to {dividend_num} times; please wait at the {dividend_num} min.")

    init = True
    for i in range(0, len(documents), text_chunk_size):
        chunk = documents[i:i + text_chunk_size]
        print(f"processing documnets from to | {i}:{i + text_chunk_size}")
        if init:
            vectorstore_chroma = Chroma.from_documents(chunk, embeddings_opennai, persist_directory=db_path)
            time.sleep(60)
            init = False
        else:
            vectorstore_chroma.add_documents(chunk)
            time.sleep(60)



chunk documents
processing documnets from to | 0:786
first init db...
processing documnets from to | 786:1572
processing documnets from to | 1572:2358


In [57]:
vectorstore_chroma.persist()

In [50]:
# vectorstore_chroma.similarity_search_with_score('習近平')
# vectorstore_chroma.similarity_search('習近平')
vectorstore_chroma.get()

{'ids': ['d4edfbf8-d1fe-11ee-a94b-26c9c135b85a',
  'fed991e8-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ecbc-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ecc6-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ce08-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ce12-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ce44-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ce58-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ce62-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ce6c-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ce80-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ce8a-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ce94-d1fe-11ee-a94b-26c9c135b85a',
  'ab94cea8-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ceb2-d1fe-11ee-a94b-26c9c135b85a',
  'ab94cebc-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ced0-d1fe-11ee-a94b-26c9c135b85a',
  'ab94ceda-d1fe-11ee-a94b-26c9c135b85a',
  'ab94cee4-d1fe-11ee-a94b-26c9c135b85a',
  'ab94cef8-d1fe-11ee-a94b-26c9c135b85a',
  'ab94cf02-d1fe-11ee-a94b-26c9c135b85a',
  'ab94cf0c-d1fe-11ee-a94b-26c9c135b85a',
  'ab94cf16-d1fe-11ee-a94b-26c9c135b85a',
  'ab94cf2a-d1fe-11ee-a94b-

In [58]:
vectorstore = Chroma(
            persist_directory=db_path, embedding_function=embeddings_opennai
        )

In [60]:
vectorstore.similarity_search('習近平')

[Document(page_content='路透社報導，與美國總統拜登會晤後，中國國家主席習近平似乎達到自己的目標，也就是在中國經濟放緩、決策蒙受質疑之際，不必太分心因應中美關係，更專注處理極為迫切的內政問題。\n習近平6年來首訪美國，和拜登在亞太經濟合作會議（APEC）高峰會場邊針對涵蓋芬太尼、軍事溝通和人工智慧（AI）等領域的合作達成共識。\n報導指出，兩名消息人士表示，這3項成果都是美國向中國尋求的結果，而不是反過來。\n但習近平似乎達到自己的目標：讓美國在政策上做出讓步，以換取合作承諾、緩和雙邊緊張局勢，從而可以更加關注經濟成長，並有機會吸引漸漸退出中國的外國投資人。\n中國經濟正在放緩，本月稍早公布的外國直接投資數據首度出現赤字。執政的共產黨也在處理引發外界質疑習近平決策的內部政治陰謀，包括外交部長和國防部長突然因不明原因遭到免職等。\n位在夏威夷的智庫太平洋論壇（Pacific Forum）兼任研究員尼爾（Alexander Neill）表示：「如果美國和中國能夠處理好分歧…這將意味習近平不必將所有注意力都轉移到（雙邊關係）上。他需要專注於極為迫切的內政。」\n確保習近平承諾中國會配合阻止芬太尼流入美國是拜登此次峰會待辦清單上的要務。高階美國官員表示，中國同意在「信任，但要查證」基礎上，追查生產芬太尼前體的特定公司。\n作為回報，美國政府16日將中國公安部物證鑑定中心從商務部貿易制裁名單中移除。該中心因涉嫌侵害維吾爾族人，於2020年被美國列入貿易制裁名單，而中國一直在爭取美國取消這項制裁。\n批評人士警告，取消對中國公安部物證鑑定中心的制裁會讓中國認為，美國的實體清單是可以談判的。批評人士也質疑，拜登政府是否會信守承諾，對新疆維族人遭中國種族滅絕一事施壓。\n共和黨領導的聯邦眾議院中國問題特別委員會發言人表示：「這降低了我們實體清單的可信度和道德權威。」', metadata={'id': '1da12d19-718f-4961-9b26-580d9002d261', 'title': '路透：習近平會拜登達目標 可專注處理內政問題', 'date': '2023/11/17', 'category': '即時國際', 'source': 'moneyudn'}),
 Document(page_content='MoneyDJ新聞 2023-11